In [1]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import svm_builder
import importlib

In [2]:
feature_number = 105

In [3]:

X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)


(393345, 105)
(393345,)


In [6]:
pca1 = PCA(n_components= 0.999).fit(X)
pca2 = PCA(n_components= 0.8).fit(X)
X_pca1 = pca1.transform(X)
X_pca2 = pca2.transform(X)
print(X_pca1.shape)
print(X_pca2.shape)

(393345, 10)
(393345, 2)


In [11]:
# model = svm_builder.train_svm_model(X_pca1, Y)

model_X = X_pca1
model_y = y
print(model_X.shape, model_y.shape)

(393345, 10) (393345,)


In [12]:
from bayes_builder import train_bayes_model
model_bayes = train_bayes_model(model_X, model_y)
model_bayes = train_bayes_model(X, y)

Training set score of Bayes: 0.4777748319356725
Testing set score of Bayes: 0.47822955154062574
Training set score of Bayes: 0.40900192851772893
Testing set score of Bayes: 0.4121131487068235


In [13]:
model_svm = svm_builder.train_svm_model(model_X, model_y)
model_svm = svm_builder.train_svm_model(X, y)

training set score: 0.4902248484606361
testing set score: 0.4928392257889563
training set score: 0.7154473906900898
testing set score: 0.7135012372461951
